# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [3]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
artifacts_path = os.getenv("ARTIFACTS_PATH", "../../artifacts")
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

✅ LLM Client configured: Using 'gemini' with model 'gemini-2.5-pro'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [4]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are an expert Database Administrator (DBA) with extensive experience in designing normalized database schemas.

Your task is to design a normalized SQL schema for a SQLite database based on the following Product Requirements Document (PRD). The schema should include tables for users and their assigned onboarding tasks, ensuring proper relationships and data integrity.

### PRD Context:
<prd>
{prd_content}
</prd>

### Requirements:
1. **Tables:**
   - Create a `users` table with the following fields:
     - `id`: Primary key, unique identifier for each user.
     - `name`: Text, the full name of the user.
     - `email`: Text, unique email address of the user.
     - `role`: Text, the role of the user (e.g., 'New Hire', 'Manager').

   - Create an `onboarding_tasks` table with the following fields:
     - `id`: Primary key, unique identifier for each task.
     - `title`: Text, the title of the task.
     - `description`: Text, a detailed description of the task.
     - `due_date`: Date, the deadline for completing the task.
     - `status`: Text, the current status of the task (e.g., 'Pending', 'Completed').
     - `user_id`: Foreign key, references the `id` field in the `users` table.

2. **Relationships:**
   - Establish a foreign key relationship between the `onboarding_tasks.user_id` field and the `users.id` field to ensure referential integrity.

3. **Normalization:**
   - Ensure the schema adheres to third normal form (3NF) to eliminate redundancy and maintain data integrity.

4. **Output:**
   - Provide only the raw SQL `CREATE TABLE` statements.
   - Ensure the SQL is compatible with SQLite syntax.

### Additional Notes:
- Use appropriate data types for each field, considering SQLite's supported types.
- Include `NOT NULL` constraints where applicable to enforce required fields.
- Add comments to the SQL statements to explain the purpose of each table and field.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
-- This script creates the database schema for the New Hire Onboarding Tool.
-- The schema is designed for SQLite and adheres to the Third Normal Form (3NF)
-- to ensure data integrity and minimize redundancy.

-- =============================================================================
-- Table: users
-- Purpose: Stores information about all users of the system, including new
-- hires, HR managers, and department managers.
-- =============================================================================
CREATE TABLE users (
    -- id: Primary Key, a unique integer identifier for each user.
    id INTEGER PRIMARY KEY AUTOINCREMENT,

    -- name: The full name of the user. This field cannot be empty.
    name TEXT NOT NULL,

    -- email: The user's unique email address, used for login and notifications.
    -- The UNIQUE constraint ensures no two users can have the same email.
    email TEXT NOT NULL UNIQUE,

    -- role: The user's role within the syst

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [5]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
You are a data specialist with expertise in generating realistic and scalable seed data for database systems. Based on the provided PRD and SQL schema, generate SQL `INSERT` statements to populate the tables with sample data for an onboarding tool.

### Context:
1. **PRD Context:**
<prd>
{prd_content}
</prd>

2. **SQL Schema:**
<schema>
{cleaned_schema}
</schema>

### Requirements:
1. **Users Table:**
   - Generate 3 users for each persona type (`New Hire`, `HR Manager`, `Department Manager`).
   - Ensure realistic and diverse names, emails, and roles.
   - Use unique email addresses to maintain data integrity.

2. **Onboarding Tasks Table:**
   - Assign 5 tasks to each user.
   - Tasks should reflect the onboarding tool's purpose (e.g., "Complete HR Paperwork", "Attend Orientation Session").
   - Include realistic `due_date` values (e.g., within 30 days of assignment).
   - Use diverse `status` values (`Pending`, `Completed`) to simulate real-world scenarios.
   - Ensure `user_id` values reference valid users from the `users` table.

3. **Implicit Data Elements:**
   - Include tasks that align with the PRD's features, such as "Compliance Training" or "Role-Specific Training Modules."
   - Use meaningful descriptions for tasks to enhance realism.
   - Ensure `due_date` values are in `YYYY-MM-DD` format to comply with SQLite standards.
   - Distribute `status` values realistically (e.g., 70% `Pending`, 30% `Completed`).
   - Include edge cases, such as tasks with `due_date` in the past or tasks assigned to multiple users.

4. **Scalability:**
   - Ensure the data can scale to larger datasets by maintaining modularity in user and task generation.
   - Use consistent naming conventions for users and tasks to support future automation.

5. **Output:**
   - Provide only the raw SQL `INSERT` statements.
   - Ensure the SQL adheres to ACID principles, maintaining data integrity and consistency.

### Additional Notes:
- Use appropriate data types and constraints as defined in the schema.
- Avoid hardcoding values; ensure the data is dynamically generated based on the schema and PRD context.
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql')
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
-- =============================================================================
-- SQL INSERT Statements for the New Hire Onboarding Tool
-- =============================================================================

-- Populating the 'users' table
-- This section inserts 9 users: 3 New Hires, 3 HR Managers, and 3 Department Managers.

INSERT INTO users (name, email, role) VALUES
('Alice Johnson', 'alice.j@company.com', 'New Hire'),
('Bob Williams', 'bob.w@company.com', 'New Hire'),
('Charlie Brown', 'charlie.b@company.com', 'New Hire'),
('Diana Prince', 'diana.p@company.com', 'HR Manager'),
('Eve Adams', 'eve.a@company.com', 'HR Manager'),
('Frank White', 'frank.w@company.com', 'HR Manager'),
('Grace Hopper', 'grace.h@company.com', 'Department Manager'),
('Henry Ford', 'henry.f@company.com', 'Department Manager'),
('Ivy Green', 'ivy.g@company.com', 'Department Manager');

-- =============================================================================


### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [6]:
def create_database(db_path, schema_path, seed_path):
    """
    Create and seed a SQLite database from SQL files.

    ### Purpose:
    This function creates a SQLite database, executes a schema SQL file to define the database structure,
    and populates it with seed data from a separate SQL file. It ensures the database adheres to ACID principles
    and is modular and scalable for future enhancements.

    ### Parameters:
    - `db_path` (str): Path to the SQLite database file. If the file exists, it will be replaced to ensure a clean slate.
    - `schema_path` (str): Path to the SQL file containing the database schema.
    - `seed_path` (str): Path to the SQL file containing the seed data.

    ### Rules:
    1. **Scalability**:
       - Ensure the function can handle larger schemas and seed data files by using efficient file handling and execution techniques.
       - Modularize the logic to allow for future extensions, such as adding support for additional SQL scripts or database types.

    2. **Modularity**:
       - Separate concerns by clearly defining the steps for schema creation, data seeding, and error handling.
       - Use helper functions where appropriate to keep the code clean and maintainable.

    3. **TOGAF Principles**:
       - Ensure the database aligns with the architecture's principles of modularity, scalability, and maintainability.
       - Design the function to support future integration with UML/SysML diagrams for database visualization.

    4. **ACID Principles**:
       - Ensure atomicity by committing all changes only after successful execution of both schema and seed data scripts.
       - Maintain consistency by validating the existence and correctness of the schema and seed files before execution.
       - Use transactions to ensure data integrity in case of errors.
       - Ensure durability by committing changes to the database file.

    ### Returns:
    - None: The function creates the database file and populates it with data, with no return value.

    ### Example Usage:
    create_database("artifacts/onboarding.db", "artifacts/schema.sql", "artifacts/seed_data.sql")
    """
    # Validate the existence of the schema file
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return

    # Remove the existing database file to ensure a clean slate
    if os.path.exists(db_path):
        os.remove(db_path)
        print(f"Removed existing database file at {db_path}")

    conn = None
    try:
        # Connect to the SQLite database (creates the file if it doesn't exist)
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # Execute the schema SQL script
        schema_sql = load_artifact(schema_path)
        if schema_sql:
            cursor.executescript(schema_sql)
            print("Database schema created successfully.")

        # Execute the seed data SQL script if it exists
        if os.path.exists(seed_path):
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted successfully.")

        # Commit all changes to ensure durability
        conn.commit()
        print("Database changes committed successfully.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # Ensure the database connection is closed to prevent resource leaks
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

Successfully connected to database at c:\Users\labadmin\Desktop\AISWE\AG-AISOFTDEV\artifacts\onboarding.db
Database schema created successfully.
Seed data inserted successfully.
Database changes committed successfully.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.